In [1]:
pip install ortools

Note: you may need to restart the kernel to use updated packages.


In [2]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import pandas as pd
import numpy as np

load C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\ortools\.libs\zlib1.dll...
load C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\ortools\.libs\abseil_dll.dll...
load C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\ortools\.libs\utf8_validity.dll...
load C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\ortools\.libs\re2.dll...
load C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\ortools\.libs\libprotobuf.dll...
load C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\ortools\.libs\highs.dll...
load C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\ortools\.libs\ortools.dll...


In [3]:
df_distance = pd.read_csv("D:distance_matrix.csv", index_col=0)

In [4]:
distance_matrix = df_distance.to_numpy()

In [5]:
data = {}

In [6]:
data['distance_matrix'] = distance_matrix
print("{:,} destinations".format(len(data['distance_matrix'][0]) - 1))

16 destinations


In [7]:
data['demands'] = [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8]

In [8]:
data

{'distance_matrix': array([[   0,  548,  776,  696,  582,  274,  502,  194,  308,  194,  536,
          502,  388,  354,  468,  776,  662],
        [ 548,    0,  684,  308,  194,  502,  730,  354,  696,  742, 1084,
          594,  480,  674, 1016,  868, 1210],
        [ 776,  684,    0,  992,  878,  502,  274,  810,  468,  742,  400,
         1278, 1164, 1130,  788, 1552,  754],
        [ 696,  308,  992,    0,  114,  650,  878,  502,  844,  890, 1232,
          514,  628,  822, 1164,  560, 1358],
        [ 582,  194,  878,  114,    0,  536,  764,  388,  730,  776, 1118,
          400,  514,  708, 1050,  674, 1244],
        [ 274,  502,  502,  650,  536,    0,  228,  308,  194,  240,  582,
          776,  662,  628,  514, 1050,  708],
        [ 502,  730,  274,  878,  764,  228,    0,  536,  194,  468,  354,
         1004,  890,  856,  514, 1278,  480],
        [ 194,  354,  810,  502,  388,  308,  536,    0,  342,  388,  730,
          468,  354,  320,  662,  742,  856],
        [ 308

In [9]:
data['vehicle_capacities'] = [15, 15, 15, 15]

In [10]:
data

{'distance_matrix': array([[   0,  548,  776,  696,  582,  274,  502,  194,  308,  194,  536,
          502,  388,  354,  468,  776,  662],
        [ 548,    0,  684,  308,  194,  502,  730,  354,  696,  742, 1084,
          594,  480,  674, 1016,  868, 1210],
        [ 776,  684,    0,  992,  878,  502,  274,  810,  468,  742,  400,
         1278, 1164, 1130,  788, 1552,  754],
        [ 696,  308,  992,    0,  114,  650,  878,  502,  844,  890, 1232,
          514,  628,  822, 1164,  560, 1358],
        [ 582,  194,  878,  114,    0,  536,  764,  388,  730,  776, 1118,
          400,  514,  708, 1050,  674, 1244],
        [ 274,  502,  502,  650,  536,    0,  228,  308,  194,  240,  582,
          776,  662,  628,  514, 1050,  708],
        [ 502,  730,  274,  878,  764,  228,    0,  536,  194,  468,  354,
         1004,  890,  856,  514, 1278,  480],
        [ 194,  354,  810,  502,  388,  308,  536,    0,  342,  388,  730,
          468,  354,  320,  662,  742,  856],
        [ 308

In [11]:
data['num_vehicles'] = 4

In [12]:
data['depot'] = 0

In [13]:
data

{'distance_matrix': array([[   0,  548,  776,  696,  582,  274,  502,  194,  308,  194,  536,
          502,  388,  354,  468,  776,  662],
        [ 548,    0,  684,  308,  194,  502,  730,  354,  696,  742, 1084,
          594,  480,  674, 1016,  868, 1210],
        [ 776,  684,    0,  992,  878,  502,  274,  810,  468,  742,  400,
         1278, 1164, 1130,  788, 1552,  754],
        [ 696,  308,  992,    0,  114,  650,  878,  502,  844,  890, 1232,
          514,  628,  822, 1164,  560, 1358],
        [ 582,  194,  878,  114,    0,  536,  764,  388,  730,  776, 1118,
          400,  514,  708, 1050,  674, 1244],
        [ 274,  502,  502,  650,  536,    0,  228,  308,  194,  240,  582,
          776,  662,  628,  514, 1050,  708],
        [ 502,  730,  274,  878,  764,  228,    0,  536,  194,  468,  354,
         1004,  890,  856,  514, 1278,  480],
        [ 194,  354,  810,  502,  388,  308,  536,    0,  342,  388,  730,
          468,  354,  320,  662,  742,  856],
        [ 308

In [14]:
# Calculate Distance between two nodes
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

In [15]:
# Get the order quantity of each node (location)
def demand_callback(from_index):
    """Returns the demand of the node."""
    # Convert from routing variable Index to demands NodeIndex.
    from_node = manager.IndexToNode(from_index)
    return data['demands'][from_node]

In [16]:
# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])


In [17]:
# Create Routing Model
routing = pywrapcp.RoutingModel(manager)

In [18]:
# Create and register a transit callback.
transit_callback_index = routing.RegisterTransitCallback(distance_callback)

In [19]:
# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

In [20]:
# Add Capacity constraint.
demand_callback_index = routing.RegisterUnaryTransitCallback(
    demand_callback)
routing.AddDimensionWithVehicleCapacity(demand_callback_index,
    0,  # null capacity slack
    data['vehicle_capacities'],  # vehicle maximum capacities
    True,  # start cumul to zero
    'Capacity')

True

In [26]:
# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
search_parameters.time_limit.FromSeconds(1)

In [27]:
# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

In [28]:
if solution:
    total_distance = 0 
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for driver {}:\n'.format(vehicle_id)
        route_distance = 0 
        route_load = 0
        while not routing.IsEnd(index):
             node_index = manager.IndexToNode(index)
             route_load += data['demands'][node_index]
             plan_output += ' {0} Parcels({1}) -> '.format(node_index, route_load)
             previous_index = index
             index = solution.Value(routing.NextVar(index))
             route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        plan_output += ' {0} Parcels({1})\n'.format(manager.IndexToNode(index),route_load)
        plan_output += 'Distance of the route: {} (m)\n'.format(route_distance)
        plan_output += 'Parcels Delivered: {} (parcels)\n'.format(route_load)
        print(plan_output)
        
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {:,} (m)'.format(total_distance))
    print('Parcels Delivered: {:,}/{:,}'.format(total_load, sum(data['demands'])))
else:
    print('No Solution')

Route for driver 0:
 0 Parcels(0) ->  7 Parcels(8) ->  3 Parcels(10) ->  4 Parcels(14) ->  1 Parcels(15) ->  0 Parcels(15)
Distance of the route: 1552 (m)
Parcels Delivered: 15 (parcels)

Route for driver 1:
 0 Parcels(0) ->  14 Parcels(4) ->  16 Parcels(12) ->  10 Parcels(14) ->  9 Parcels(15) ->  0 Parcels(15)
Distance of the route: 1552 (m)
Parcels Delivered: 15 (parcels)

Route for driver 2:
 0 Parcels(0) ->  12 Parcels(2) ->  11 Parcels(3) ->  15 Parcels(11) ->  13 Parcels(15) ->  0 Parcels(15)
Distance of the route: 1552 (m)
Parcels Delivered: 15 (parcels)

Route for driver 3:
 0 Parcels(0) ->  8 Parcels(8) ->  2 Parcels(9) ->  6 Parcels(13) ->  5 Parcels(15) ->  0 Parcels(15)
Distance of the route: 1552 (m)
Parcels Delivered: 15 (parcels)

Total distance of all routes: 6,208 (m)
Parcels Delivered: 60/60
